# Text Autocompletion Project — From LSTM to Transformers

This notebook-style Python script implements the complete pipeline required by the
project brief. It is structured as a sequence of Markdown explanations followed by
executable cells. The scope and goals mirror the assignment:

- Prepare and normalize a corpus of short posts from a line-delimited text file.
- Build and train a compact next-token LSTM language model suitable for constrained
  devices.
- Measure quality using ROUGE-1 and ROUGE-2 on held-out text completions (predicting
  the last quarter of each text from the first three quarters).
- Benchmark against a pretrained transformer baseline (DistilGPT2) using a simple
  generation pipeline.
- Compare results and provide a concise conclusion.

The machine learning formulation is next-token prediction: given tokens
$(w_1, \dots, w_n)$, the model estimates $P(w_{n+1} \mid w_{\le n})$ and we
iteratively generate until an end token or a maximum length is reached.

Datasets are split deterministically into train/validation/test, tokenized at the
word level with special tokens `<pad>`, `<unk>`, `<bos>`, `<eos>`, and converted into
shifted input/target tensors for supervised learning.

Evaluation follows the assignment’s rule of using the first 75% tokens of each text as
the prefix and comparing generated completions to the remaining 25% via ROUGE.

In [ ]:
from __future__ import annotations

import logging
from pathlib import Path

import matplotlib.pyplot as plt

from src.config_utils import LSTMConfig, load_config_model
from src.data_utils import load_processed_texts, preprocess_dataset, split_and_persist_texts
from src.eval_lstm import evaluate_on_texts
from src.eval_transformer_pipeline import evaluate_gpt2_on_texts
from src.lstm_model import LSTMLanguageModel
from src.lstm_train import create_optimizer_scheduler, save_checkpoint, train_epoch, valid_epoch
from src.next_token_dataset import create_next_token_dataloaders
from src.project_utils import initialize_runtime, print_environment
from src.summary import print_comparison, print_scores, show_examples
from src.tokenizer_vocab import WordVocab, build_supervised_tensors

logging.basicConfig(level=logging.INFO)

## Configuration

We load a typed configuration object that centralizes paths, data limits, model
hyperparameters, training settings, and generation defaults. This ensures the entire
experiment is reproducible and easy to tweak without touching the code.

In [ ]:
CFG = load_config_model()

## Runtime Initialization

We set a fixed seed for reproducibility, select the device, optionally limit CPU
threads, and print the environment. Keeping runs deterministic is crucial for fair
comparisons between the LSTM and the transformer baseline.

In [ ]:
ctx = initialize_runtime(seed=42, limit_cpu_threads=True)
device = ctx.device
print_environment(device)

## Data Preparation (Normalization and Filtering)

We ingest a line-delimited UTF-8 text file (`tweets.txt`, one text per line) and produce a single-column `text` CSV after:
- Lowercasing and basic normalization
- Removing URLs, mentions, and optional emojis
- Collapsing excessive whitespace
- Filtering by minimal character count and maximal token length

This creates a clean, consistent corpus for modeling.

In [ ]:
processed_csv = preprocess_dataset(
    raw_data_path=CFG.data.raw_data_path,
    out_path=CFG.data.processed_path,
    dev_limit=CFG.data.dev_limit,
    min_chars=CFG.preprocessing.min_chars,
    max_tokens=CFG.preprocessing.max_tokens,
)
print(f"Processed dataset written to: {processed_csv}")

## Load Processed Texts

We read the normalized single-column corpus. For fast iteration, we may limit the
number of examples during development via the config.

In [ ]:
texts = load_processed_texts(CFG.data.processed_path, limit=CFG.data.dev_limit)
print(f"Loaded {len(texts):,} texts from {CFG.data.processed_path}")
print("Sample:", texts[:3])

## Deterministic Split: Train / Validation / Test

We perform a seeded shuffle and split into train, validation, and test subsets, then
persist each split to disk. This mirrors how production data pipelines audit and
version data artifacts.

In [ ]:
train_texts, val_texts, test_texts, _paths = split_and_persist_texts(
    texts=texts,
    train_path=CFG.data.train_path,
    val_path=CFG.data.val_path,
    test_path=CFG.data.test_path,
    train_frac=CFG.split.train_frac,
    val_frac=CFG.split.val_frac,
    seed=CFG.split.seed,
    dev_limit=CFG.data.dev_limit,
)

## Tokenizer and Vocabulary

We build a compact word-level vocabulary with special tokens `<pad>`, `<unk>`, `<bos>`,
and `<eos>`. The tokenizer uses a configurable token pattern and frequency threshold to
control vocabulary size, trading off coverage and memory footprint.

In [ ]:
vocab = WordVocab.build(
    texts,
    special_tokens={
        "pad": CFG.special_tokens.pad,
        "unk": CFG.special_tokens.unk,
        "bos": CFG.special_tokens.bos,
        "eos": CFG.special_tokens.eos,
    },
    max_vocab_size=CFG.tokenizer.max_vocab_size,
    token_pattern=CFG.tokenizer.token_pattern,
    min_freq=CFG.tokenizer.min_freq,
)
print("Vocab size (including specials):", len(vocab.id_to_token))
print("Special IDs:", {k: getattr(vocab, f"{k}_id") for k in ("pad", "unk", "bos", "eos")})

## Supervised Tensors for Next-Token Prediction

We convert each text into three tensors per split:
- `input_ids`: token IDs including `<bos>` and `<eos>`, padded to a fixed length
- `target_ids`: `input_ids` shifted left by one position
- `attention_mask`: marks valid (non-pad) positions for loss masking

This enables standard cross-entropy training for next-token prediction.

In [ ]:
train_input_ids, train_target_ids, train_attention_mask = build_supervised_tensors(
    train_texts, vocab, max_len=CFG.sequence.max_length, device=device
)
val_input_ids, val_target_ids, val_attention_mask = build_supervised_tensors(
    val_texts, vocab, max_len=CFG.sequence.max_length, device=device
)
test_input_ids, test_target_ids, test_attention_mask = build_supervised_tensors(
    test_texts, vocab, max_len=CFG.sequence.max_length, device=device
)

print("Splits:", len(train_texts), len(val_texts), len(test_texts))
print("Train tensors:", train_input_ids.shape, train_target_ids.shape, train_attention_mask.shape)
print("Val tensors:", val_input_ids.shape, val_target_ids.shape, val_attention_mask.shape)
print("Test tensors:", test_input_ids.shape, test_target_ids.shape, test_attention_mask.shape)

## DataLoaders

We wrap the tensors into `DataLoader`s for mini-batch training and evaluation, shuffling
only the training split. This keeps validation/test metrics comparable across runs.

In [ ]:
train_loader, val_loader, test_loader = create_next_token_dataloaders(
    train_tensors=(train_input_ids, train_target_ids, train_attention_mask),
    val_tensors=(val_input_ids, val_target_ids, val_attention_mask),
    test_tensors=(test_input_ids, test_target_ids, test_attention_mask),
    cfg=CFG,
    device=device,
)

## LSTM Language Model

We instantiate a compact LSTM-based language model configured for mobile-friendly
footprint. The model predicts the next token distribution at each time step.

In [ ]:
config = LSTMConfig(
    num_embeddings=len(vocab.id_to_token),
    embedding_dim=CFG.model.lstm.embedding_dim,
    hidden_size=CFG.model.lstm.hidden_size,
    num_layers=CFG.model.lstm.num_layers,
    dropout=CFG.model.lstm.dropout,
    pad_token_id=vocab.pad_id,
)
model = LSTMLanguageModel(config).to(device)

### Parameter Count

We report total and trainable parameters to gauge capacity and memory cost. This helps
justify design choices when targeting limited-resource environments.

In [ ]:
num_params = sum(p.numel() for p in model.parameters())
num_params_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model params: total={num_params:,}, trainable={num_params_trainable:,}")

## Optimization Setup

We use standard cross-entropy training with an optimizer configured from the config and
an optional StepLR scheduler. Gradient clipping is applied during the training step to
stabilize optimization.

In [ ]:
optimizer, scheduler = create_optimizer_scheduler(
    model,
    learning_rate=CFG.training.learning_rate,
    weight_decay=CFG.training.weight_decay,
    use_scheduler=CFG.scheduler.use,
    step_size=CFG.scheduler.step_size,
    gamma=CFG.scheduler.gamma,
)

## Training Loop and Loss Curves

We train for a small number of epochs, recording train/validation loss per epoch. A
loss curve is saved to `data/loss_curve.png` for quick inspection of convergence and
generalization.

In [ ]:
print("Starting training...")
train_losses: list[float] = []
val_losses: list[float] = []
for epoch in range(1, (CFG.training.epochs or 1) + 1):
    tr = train_epoch(
        model,
        train_loader,
        optimizer,
        device,
        max_grad_norm=CFG.training.max_grad_norm,
        progress_desc=f"epoch {epoch}/{CFG.training.epochs or 1}",
    )
    va = valid_epoch(model, val_loader, device)
    if scheduler is not None:
        scheduler.step()
    train_losses.append(tr)
    val_losses.append(va)
    print(f"epoch {epoch}: train={tr:.4f} val={va:.4f}")

if len(train_losses) > 0:
    epochs_axis = list(range(1, len(train_losses) + 1))
    plt.figure(figsize=(7, 4))
    plt.plot(epochs_axis, train_losses, label="Train loss")
    plt.plot(epochs_axis, val_losses, label="Val loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training/Validation Loss")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()
    plt.close()

## Checkpointing

We save a minimal checkpoint containing the model parameters and the vocabulary. This
is sufficient to run inference and evaluation later without retraining.

In [ ]:
save_checkpoint(model, vocab, state_path=str(Path("data") / "lstm_state.pt"), vocab_path=str(Path("data") / "vocab.pt"))
print("Saved model state and vocab to data/.")

## Evaluation: LSTM on Test Split

Using the assignment protocol, we evaluate ROUGE-1/2 (precision, recall, F1) by
generating the last quarter of each text from its first three quarters and comparing to
the reference continuation.

In [ ]:
test_scores = evaluate_on_texts(
    model,
    test_texts,
    vocab=vocab,
    device=device,
    max_new_tokens=CFG.generation.max_new_tokens,
    max_samples=CFG.evaluation.test_samples,
)
print("LSTM test ROUGE:", {k: round(v, 4) for k, v in test_scores.items()})

## Evaluation: DistilGPT2 Baseline

We benchmark a pretrained `distilgpt2` via the transformers text-generation pipeline on
the same protocol to contextualize the LSTM’s performance.

In [ ]:
gpt2_test_scores = evaluate_gpt2_on_texts(
    texts=test_texts,
    vocab=vocab,
    device=device,
    model_name=CFG.gpt2.model_name,
    max_new_tokens=CFG.generation.max_new_tokens,
    max_samples=CFG.gpt2.eval_samples,
    token_pattern=CFG.tokenizer.token_pattern,
)
print("GPT2 Test ROUGE:", {k: round(v, 4) for k, v in gpt2_test_scores.items()})

## Results Summary and Qualitative Examples

We print ROUGE summaries for both models, provide a side-by-side comparison, and show
qualitative completions for a handful of train/validation/test samples to build
intuition beyond aggregate metrics.

In [ ]:
print_scores("LSTM Test ROUGE (P/R/F1):", test_scores)
print_scores("DistilGPT2 Test ROUGE (P/R/F1):", gpt2_test_scores)
print_comparison("LSTM (Test)", test_scores, "DistilGPT2 (Test)", gpt2_test_scores)

In [ ]:
show_examples(model, train_texts, vocab, device, CFG.generation.max_new_tokens, k=3)
show_examples(model, val_texts, vocab, device, CFG.generation.max_new_tokens, k=3)

In [ ]:
show_examples(model, test_texts, vocab, device, CFG.generation.max_new_tokens, k=8)

## Conclusion

We select the better model by ROUGE-1 F1 on the test split and print the decision. In
practice, the transformer baseline often wins on quality, while the LSTM may be
preferable under strict memory and latency constraints. The final choice depends on the
target device budget and user experience requirements.

In [ ]:
gpt2_r1 = gpt2_test_scores.get("rouge1_f1")
lstm_r1 = test_scores.get("rouge1_f1")
gpt2_test = float(gpt2_r1) if isinstance(gpt2_r1, int | float) and gpt2_r1 == gpt2_r1 else float("nan")
lstm_test = float(lstm_r1) if isinstance(lstm_r1, int | float) and lstm_r1 == lstm_r1 else float("nan")
better = "DistilGPT2" if gpt2_test > lstm_test else "LSTM"
print(f"Conclusion: {better} performed better on ROUGE-1 F1 on the test split.")